In [ ]:
!pip install fastapi uvicorn streamlit pyngrok pandas requests prophet tsfresh scikit-learn matplotlib    --quiet

In [ ]:
from pyngrok import ngrok

# Please replace 'YOUR_NGROK_AUTHTOKEN' with your actual ngrok authentication token
ngrok.set_auth_token('38D0P5dopGSihiwuvxUSqExgpzY_53JxC4N2ab8HiPSsbwHUh')

In [ ]:
%%writefile backend.py

from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
import pandas as pd
import numpy as np
import traceback

from prophet import Prophet
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

app = FastAPI(title="FitPulse – Health Anomaly Backend")

# ---------------- GLOBAL STATE ----------------
CLEAN_DF = None
FEATURE_DF = None
ALERTS_DF = None

# ---------------- ERROR HANDLER ----------------
@app.exception_handler(Exception)
async def global_exception_handler(request, exc):
    trace = traceback.format_exc()
    print(trace)
    return JSONResponse(
        status_code=500,
        content={"error": "Backend crashed", "trace": trace}
    )

# ==================================================
# MODULE 1 – PREPROCESSING
# ==================================================
@app.post("/preprocess")
def preprocess(file: UploadFile = File(...)):
    global CLEAN_DF

    df = pd.read_csv(file.file)

    REQUIRED = ["Id", "date", "avg_heart_rate", "daily_steps", "hours_sleep"]
    missing = [c for c in REQUIRED if c not in df.columns]
    if missing:
        return JSONResponse(status_code=400, content={"error": f"Missing {missing}"})

    df = df.rename(columns={
        "Id": "user_id",
        "avg_heart_rate": "heart_rate",
        "daily_steps": "steps",
        "hours_sleep": "sleep"
    })

    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date"])

    for col in ["heart_rate", "steps", "sleep"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    df["heart_rate"].fillna(df["heart_rate"].median(), inplace=True)
    df["steps"].fillna(0, inplace=True)
    df["sleep"].fillna(df["sleep"].median(), inplace=True)

    df = (
        df.set_index("date")
        .groupby("user_id")[["heart_rate", "steps", "sleep"]]
        .resample("D")
        .mean()
        .reset_index()
    )

    CLEAN_DF = df
    df.to_csv("clean_data.csv", index=False)

    return {"status": "success", "rows": len(df)}

# ==================================================
# MODULE 2 – FEATURE EXTRACTION + MODELING
# ==================================================
@app.post("/module2")
def module2():
    global CLEAN_DF, FEATURE_DF

    if CLEAN_DF is None:
        return JSONResponse(status_code=400, content={"error": "Run Module 1 first"})

    df = CLEAN_DF.sort_values(["user_id", "date"])

    # ---------- Feature Extraction ----------
    df["hr_7d_mean"] = df.groupby("user_id")["heart_rate"].transform(
        lambda x: x.rolling(7, min_periods=1).mean()
    )
    df["hr_7d_std"] = df.groupby("user_id")["heart_rate"].transform(
        lambda x: x.rolling(7, min_periods=1).std().fillna(0)
    )
    df["steps_7d_mean"] = df.groupby("user_id")["steps"].transform(
        lambda x: x.rolling(7, min_periods=1).mean()
    )

    FEATURE_DF = df.copy()

    # ---------- Prophet ----------
    plot = []
    try:
        p_df = (
            df.groupby("date")["heart_rate"]
            .mean()
            .reset_index()
            .rename(columns={"date": "ds", "heart_rate": "y"})
        )
        if len(p_df) > 10:
            m = Prophet()
            m.fit(p_df)
            f = m.predict(p_df)
            p_df["yhat"] = f["yhat"]
            plot = p_df.tail(30).to_dict("records")
    except:
        pass

    # ---------- DBSCAN ----------
    X = FEATURE_DF[["heart_rate", "steps", "sleep", "hr_7d_mean"]].fillna(0)
    X = StandardScaler().fit_transform(X)

    db = DBSCAN(eps=1.2, min_samples=5)
    FEATURE_DF["cluster"] = db.fit_predict(X)
    FEATURE_DF["cluster_anomaly"] = FEATURE_DF["cluster"] == -1

    FEATURE_DF.to_csv("feature_data.csv", index=False)

    return {"status": "success", "sample_plot": plot}

# ==================================================
# MODULE 3 – ANOMALY DETECTION
# ==================================================
@app.post("/module3")
def module3():
    global FEATURE_DF, ALERTS_DF

    if FEATURE_DF is None:
        return JSONResponse(status_code=400, content={"error": "Run Module 2 first"})

    rows = []

    for _, r in FEATURE_DF.iterrows():
        if r["heart_rate"] > 120:
            rows.append((r["user_id"], r["date"], "heart_rate_high"))
        if r["heart_rate"] < 40:
            rows.append((r["user_id"], r["date"], "heart_rate_low"))
        if r["sleep"] < 4 or r["sleep"] > 12:
            rows.append((r["user_id"], r["date"], "sleep_abnormal"))
        if r["cluster_anomaly"]:
            rows.append((r["user_id"], r["date"], "cluster_outlier"))

    df = pd.DataFrame(rows, columns=["user_id", "date", "metric"])

    alerts = (
        df.groupby(["user_id", "metric"])
        .agg(count=("date", "count"))
        .reset_index()
    )

    alerts["severity"] = alerts["count"].apply(
        lambda x: "High" if x >= 5 else "Medium" if x >= 3 else "Low"
    )

    ALERTS_DF = alerts
    alerts.to_csv("module3_alerts.csv", index=False)

    return {"status": "success", "alerts": alerts.to_dict("records")}

# ==================================================
# MODULE 4 – INSIGHTS
# ==================================================
@app.post("/module4")
def module4():
    if ALERTS_DF is None or ALERTS_DF.empty:
        return JSONResponse(status_code=400, content={"error": "Run Module 3 first"})

    insights = [
        {
            "user_id": r["user_id"],
            "severity": r["severity"],
            "insight": f"User {r['user_id']} has {r['severity']} risk due to {r['metric']}"
        }
        for _, r in ALERTS_DF.iterrows()
    ]

    return {"status": "success", "insights": insights}


Overwriting backend.py


In [ ]:
from pyngrok import ngrok
import uvicorn
import threading

# Kill any existing ngrok processes to avoid 'simultaneous sessions' error
!pkill -f ngrok

public_url = ngrok.connect(8000)
print("FastAPI URL:", public_url)

threading.Thread(
    target=uvicorn.run,
    kwargs={"app": "backend:app", "host": "0.0.0.0", "port": 8000}
).start()

FastAPI URL: NgrokTunnel: "https://chrematistic-commonsensical-mellie.ngrok-free.dev" -> "http://localhost:8000"


In [ ]:
%%writefile app.py
import streamlit as st
import requests
import pandas as pd
import plotly.express as px

BACKEND_URL = "http://localhost:8000"

st.set_page_config(
    page_title="FitPulse – Health Anomaly Detection",
    layout="wide"
)

# HEADER
st.title(" FitPulse Health Monitoring Dashboard")
st.caption("AI-powered detection of unusual health patterns from fitness device data")
st.divider()

# SIDEBAR – FILE UPLOAD
st.sidebar.header("📂 Upload Fitness Data")

uploaded_file = st.sidebar.file_uploader(
    "Upload CSV file",
    type=["csv"]
)

if uploaded_file:
    with st.spinner("Preparing data..."):
        res = requests.post(
            f"{BACKEND_URL}/preprocess",
            files={"file": uploaded_file}
        )

    if res.status_code == 200:
        st.sidebar.success("Data processed successfully")
        st.sidebar.metric("Records Loaded", res.json()["rows"])
    else:
        st.sidebar.error(res.json()["error"])

# LOAD CLEAN DATA
@st.cache_data
def load_clean():
    return pd.read_csv("clean_data.csv", parse_dates=["date"])

try:
    df = load_clean()
except:
    df = None

# FILTERS
st.subheader(" Filters")

if df is not None:
    c1, c2, c3 = st.columns(3)

    with c1:
        users = ["All"] + sorted(df["user_id"].unique().tolist())
        selected_user = st.selectbox("User", users)

    with c2:
        start_date, end_date = st.date_input(
            "Date Range",
            value=[df["date"].min(), df["date"].max()]
        )

    with c3:
        metric = st.selectbox(
            "Health Metric",
            ["Heart Rate", "Sleep", "Steps"]
        )

    if selected_user != "All":
        df = df[df["user_id"] == selected_user]

    df = df[
        (df["date"] >= pd.to_datetime(start_date)) &
        (df["date"] <= pd.to_datetime(end_date))
    ]

else:
    st.info("Upload data to activate dashboard")
    st.stop()

st.divider()

# TABS
tab1, tab2, tab3, tab4 = st.tabs([
    " Overview",
    " Pattern Analysis",
    " Anomalies",
    " Insights & Reports"
])

# TAB 1 – OVERVIEW
with tab1:
    st.subheader(" Data Overview")

    c1, c2, c3 = st.columns(3)
    c1.metric("Users", df["user_id"].nunique())
    c2.metric("Days", df["date"].nunique())

    if metric == "Heart Rate":
        c3.metric("Avg HR", round(df["heart_rate"].mean(), 1))
        y_col = "heart_rate"
    elif metric == "Sleep":
        c3.metric("Avg Sleep (hrs)", round(df["sleep"].mean(), 2))
        y_col = "sleep"
    else:
        c3.metric("Avg Steps", int(df["steps"].mean()))
        y_col = "steps"

    fig = px.line(
        df,
        x="date",
        y=y_col,
        color="user_id",
        title=f"{metric} Trend"
    )
    st.plotly_chart(fig, use_container_width=True)

    st.dataframe(df.head(50), use_container_width=True)

# TAB 2 – PATTERN ANALYSIS
with tab2:
    st.subheader(" Health Pattern Analysis")

    if st.button("Run Analysis"):
        with st.spinner("Running analysis..."):
            res = requests.post(f"{BACKEND_URL}/module2")

        if res.status_code == 200:
            st.success("Analysis completed")

            # -------- Heart Rate Trend --------
            hr_plot = res.json().get("heart_rate_plot", [])
            if hr_plot:
                pdf = pd.DataFrame(hr_plot)
                fig = px.line(
                    pdf,
                    x="ds",
                    y=["y", "yhat"],
                    title="Heart Rate vs Baseline"
                )
                st.plotly_chart(fig, use_container_width=True)

            # -------- Sleep Trend --------
            sleep_plot = res.json().get("sleep_plot", [])
            if sleep_plot:
                sdf = pd.DataFrame(sleep_plot)
                fig = px.line(
                    sdf,
                    x="date",
                    y="sleep",
                    title="Sleep Duration Trend"
                )
                st.plotly_chart(fig, use_container_width=True)

            # -------- Cluster Scatter --------
            cluster_plot = res.json().get("cluster_plot", [])
            if cluster_plot:
                cdf = pd.DataFrame(cluster_plot)
                cdf["type"] = cdf["cluster_anomaly"].map(
                    {True: "Outlier", False: "Normal"}
                )

                fig = px.scatter(
                    cdf,
                    x="pca_x",
                    y="pca_y",
                    color="type",
                    title="Behavior Clustering (DBSCAN + PCA)",
                    opacity=0.7
                )
                st.plotly_chart(fig, use_container_width=True)

        else:
            st.error(res.json()["error"])

# TAB 3 – ANOMALIES
with tab3:
    st.subheader(" Detected Health Anomalies")

    if st.button("Detect Anomalies"):
        with st.spinner("Detecting anomalies..."):
            res = requests.post(f"{BACKEND_URL}/module3")

        if res.status_code == 200:
            alerts = pd.DataFrame(res.json()["alerts"])

            if selected_user != "All":
                alerts = alerts[alerts["user_id"] == selected_user]

            if not alerts.empty:
                st.dataframe(alerts, use_container_width=True)

                fig = px.bar(
                    alerts,
                    x="metric",
                    y="count",
                    color="severity",
                    title="Anomaly Summary"
                )
                st.plotly_chart(fig, use_container_width=True)
            else:
                st.info("No anomalies detected")
        else:
            st.error(res.json()["error"])

# TAB 4 – INSIGHTS & REPORTS
with tab4:
    st.subheader(" Health Insights")

    if st.button("Generate Insights"):
        with st.spinner("Generating insights..."):
            res = requests.post(f"{BACKEND_URL}/module4")

        if res.status_code == 200:
            for i in res.json()["insights"]:
                st.warning(
                    f" User {i['user_id']} — {i['severity']} Risk\n\n"
                    f"{i['insight']}"
                )
        else:
            st.error(res.json()["error"])

    st.divider()
    st.subheader("Downloads")

    try:
        st.download_button(
            "Download Clean Data",
            open("clean_data.csv", "rb"),
            file_name="clean_data.csv"
        )

        st.download_button(
            "Download Alerts Report",
            open("module3_alerts.csv", "rb"),
            file_name="health_alerts.csv"
        )
    except:
        st.info("Reports available after analysis")


Writing app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
from pyngrok import ngrok
streamlit_url = ngrok.connect(8501)
print("Streamlit public URL:", streamlit_url)

Streamlit public URL: NgrokTunnel: "https://chrematistic-commonsensical-mellie.ngrok-free.dev" -> "http://localhost:8501"


In [ ]:
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0



2026-01-13 15:41:05.738 Port 8501 is already in use


In [ ]:
#!pkill -f ngrok
#!pkill -f uvicorn